In [3]:
import re
import urllib
import pandas as pd
import pygsheets
import copy
from datetime import datetime
import numpy as np

#pull the html/xml data from site into a giant string
url=('https://www.teletrader.com/quickbar/Henry_Hub_Natural_Gas_Futures_ETH_Energy_NYMEX_Futures')

items=re.compile(r"<tr class.*?</tr>")
answeritems=[]
allcontracts=[]
today=datetime.today()
nowtime=datetime.now()

f = urllib.request.urlopen(url)

contents = str(f.readlines())

f.close()

#parse the giant string of site data with regular expressions to pull out the pricing data
#the product of thiese lines is a list of lists 
answeritems=items.findall(contents)[8:]

for i in answeritems:
    tts=re.compile(r'-\d\d\d\d\d\d\d\d')
    monthyear=re.compile(r'-\d-\d\d')
    twomonthyear=re.compile(r'-\d\d-\d\d')
    price=re.compile(r'delayed">.*?<')

    ttsnumber=str(set(tts.findall(i)))[3:11]
    monthy=(str(set(monthyear.findall(i)))[3:7]+str(set(twomonthyear.findall(i)))[3:8]).replace("()","").replace("-","-01-")
    contractprice=float(str(set(price.findall(i)))[11:].replace("<'}",""))
    strip=int(monthy[:2].replace("-",""))
    stripm=int(monthy[:2].replace("-",""))

    if strip == 11 or strip==12:
        strip="Nov"+str(monthy[-2:])+"-Mar"+str(int(monthy[-2:])+1)
    elif strip == 1 or strip==2 or strip==3:
        strip="Nov"+str(int(monthy[-2:])-1)+"-Mar"+str(monthy[-2:])
    else:
        strip="Apr"+str(monthy[-2:])+"-Oct"+str(monthy[-2:])
   
    allcontracts.append((ttsnumber,monthy,contractprice,strip))

#print(allcontracts)

#Convert the list into a Pandas dataframe
#give the columns some names
df=pd.DataFrame(allcontracts)
df.columns=["TTS","Contract Month","Price","Strip"]

#Create Average Strip Table
dfstrips=copy.deepcopy(df)
TradeDateTime=[]

dfstrips.drop("TTS",1, inplace=True)
dfstrips.drop("Contract Month",1,inplace=True)

dfstrips=dfstrips.groupby(['Strip']).mean()

TradeDate=[today]*len(dfstrips)
TradeTime=[datetime.now()]*len(dfstrips)

Index=np.linspace(0,len(dfstrips)-1,len(dfstrips))

dfstrips['Trade Date']=TradeDate
dfstrips['Trade Time']=TradeTime
dfstrips=dfstrips.reset_index()
dfstrips.columns=['1','','','']

#access Google Sheets using the Google API JSON file
gc = pygsheets.authorize(service_file='C:\\Users\\Dmitry\\Documents\\Consulting Projects\\NYMEX Charts\\creds2.json')
sh = gc.open('NYMEX_Charts')

#find the first empty row in the 1st Google Sheet
insertrow1=0
wks = sh[0]
sh = gc.open('NYMEX_Charts')
rowcount=0
allrows=wks.get_all_values()

while allrows[rowcount][0]!='':
    rowcount=rowcount+1

insertrow1=rowcount+1

#update sheet1 with Current Curve
TradeDatedf=[today]*(len(df))
TradeTimedf=[datetime.now()]*(len(df))
df['Trade Date']=TradeDatedf
df['Trade Time']=TradeTimedf

wks = sh[0]
wks.set_dataframe(df,(insertrow1,1))

#find the first empty row in the 2nd Google Sheet
insertrow=0
wks = sh[1]
sh = gc.open('NYMEX_Charts')
rowcount=0
allrows=wks.get_all_values()

while allrows[rowcount][0]!='':
    rowcount=rowcount+1

insertrow=rowcount+1

#update sheet2 with Strips
wks = sh[1]
wks.set_dataframe(dfstrips,(insertrow,1))

#print(df)
